In [2]:
from parsel import Selector
from tqdm import tqdm
import pandas as pd

In [13]:
arquivo = open('pagina_de_livro25.txt', 'r', encoding="utf-8")

In [14]:
html = arquivo.read()

In [15]:
type(html)

str

In [16]:
pagina = Selector(text=html)

### Titulos:

In [17]:
pagina.xpath('//span[@id="productTitle"]/text()').get().strip()

'Reconhecimento de Padrões: um Estudo Dirigido'

#### Autores:

In [18]:
pagina.xpath('//span[@class="author notFaded"]/a/text()').getall()

['Rogério Galante Negri']

#### Ano ou tipo:

In [22]:
pagina.xpath('//span[@id="productSubtitle"]/text()').getall()[0].strip()[:11].strip()

'Capa comum'

#### Avaliação:

In [23]:
pagina.xpath('//a[@class="a-popover-trigger a-declarative"]/span[@class="a-size-base a-color-base"]/text()').get()


' 4,7 '

#### Total avaliação:

In [24]:
pagina.xpath('//span[@id="acrCustomerReviewText"]/text()').get()

'74 avaliações de clientes'

#### Descrição:

In [25]:
def descricao(valor):
    """Apos pegar um conjunto de dados 
    possives de rotular ele retorna um dicionário - Para carrocel de dados"""
    aux = pagina.xpath('//ol[@class="a-carousel"]//li[@class="a-carousel-card rpi-carousel-attribute-card"]//span/text()').getall()
    descricao = {}
    for index, i in enumerate(aux):
        if index % 2 == 0:
            descricao[i] = aux[index+1]
    if valor in descricao.keys():
        return descricao[valor]
    elif "dict" in valor:
        return descricao
    else:
        return "vazio"

In [26]:
descricao('dict')

{'ISBN-10': '6555061634',
 'ISBN-13': '978-6555061635',
 'Edição': '1ª',
 'Editora': 'Blucher',
 'Data da publicação': '9 dezembro 2021',
 'Idioma': 'Português',
 'Dimensões': '17 x 1 x 24 cm',
 'Número de páginas': '276 páginas'}

In [31]:
final = 579
autores = []
titulos = []
avaliacoes = []
total_avaliacoes = []
paginas = []
ano = []
editora = []
tipo = []

for i in tqdm(range(final)):
    arquivo = open(f'pagina_de_livro{i}.txt', 'r', encoding="utf-8")
    html = arquivo.read()
    pagina = Selector(text=html)
    autores.append(pagina.xpath('//span[@class="author notFaded"]/a/text()').getall())
    titulos.append(pagina.xpath('//span[@id="productTitle"]/text()').get().strip())
    avaliacoes.append(pagina.xpath('//a[@class="a-popover-trigger a-declarative"]/span[@class="a-size-base a-color-base"]/text()').get())
    total_avaliacoes.append(pagina.xpath('//span[@id="acrCustomerReviewText"]/text()').get())
    paginas.append(descricao('Número de páginas'))
    ano.append(descricao('Data da publicação'))
    editora.append(descricao('Editora'))
    tipo.append(pagina.xpath('//span[@id="productSubtitle"]/text()').getall())


100%|████████████████████████████████████████████████████████████████████████████████| 579/579 [00:58<00:00,  9.95it/s]


# base Pre Processada:

In [35]:
df1 = pd.DataFrame(titulos).rename(columns={0:'Título'})
df2 = pd.DataFrame(autores).rename(columns={0:'Autor'})
df3 = pd.DataFrame(avaliacoes).rename(columns={0:'Nota'})
df4 = pd.DataFrame(total_avaliacoes).rename(columns={0:'Total de Avaliações'})
df5 = pd.DataFrame(paginas).rename(columns={0:'Nº de Páginas'})
df6 = pd.DataFrame(ano).rename(columns={0:'Data'})
df7 = pd.DataFrame(editora).rename(columns={0:'Editora'})
df8 = pd.DataFrame(tipo).rename(columns={0:"Tipo"})

In [36]:
data = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
data.rename(columns={1:"Co Autor", 2:'Co-Autor..'}, inplace=True)



In [37]:
data

,Título,Autor,Co Autor,Co-Autor..,Nota,Total de Avaliações,Nº de Páginas,Data,Editora,Tipo
0,Introdução À Algoritmia e Programação com Python,Filipe Portela,Tiago C. Pereira,None,"1,0",1 classificação,384 páginas,21 novembro 2023,Fca,Capa comum – 21 novembro 2023
1,Python: guia básico para iniciantes,Francisco Jonas,None,None,"5,0",1 classificação,62 páginas,3 dezembro 2023,vazio,eBook Kindle
2,Código limpo: habilidades práticas do Agile so...,Robert C. Martin,None,None,"4,9",6.501 avaliações de clientes,425 páginas,8 setembro 2009,Alta Books,"Capa comum – Edição padrão, 8 setembro 2009"
3,Python descomplicado com o Chatgpt.,Clayton Costa,None,None,"4,4",4 avaliações de clientes,27 páginas,15 março 2024,vazio,eBook Kindle
4,"A próxima onda: Inteligência artificial, poder...",Mustafa Suleyman,Michael Bhaskar,None,"4,6",134 avaliações de clientes,420 páginas,16 outubro 2023,Record,Capa comum – 16 outubro 2023
...,...,...,...,...,...,...,...,...,...,...
574,Linguagens de programação orientadas a objetos...,Fonte: Wikipedia,None,None,None,None,46 páginas,26 junho 2011,"Books LLC, Wiki Series",Capa comum – 26 junho 2011
575,"Software por licença: Licenças de Software, So...",Fonte: Wikipedia,None,None,None,None,226 páginas,30 junho 2011,"Books LLC, Wiki Series",Capa comum – 30 junho 2011
576,Humoristas do Reino Unido: Humoristas da Ingla...,Fonte: Wikipedia,None,None,None,None,30 páginas,26 junho 2011,"Books LLC, Wiki Series",Capa comum – 26 junho 2011
577,"Linguagens de script: Python, Euphoria, JavaSc...",Fonte: Wikipedia,None,None,"1,0",1 classificação,50 páginas,25 junho 2011,"Books LLC, Wiki Series",Capa comum – 25 junho 2011


In [38]:
df8 = pd.DataFrame(tipo).rename(columns={0:"Tipo"})
